In [1]:
import chess
import ipywidgets as widgets
import chess.pgn
import numpy as np
import io
import sys


if sys.platform=='win32':
    STOCKFISHPATH=r'c:/gitproj/stockfish/stockfish-windows-x86-64-avx2.exe'
    import nest_asyncio
    nest_asyncio.apply()
else:
    STOCKFISHPATH='/home/jovyan/stockfish/src/stockfish'

engine = chess.engine.SimpleEngine.popen_uci(STOCKFISHPATH)

from ipywidgets import interact, interactive, fixed, interact_manual, SelectMultiple, Combobox, HBox, VBox
import ipywidgets as widgets
from collections import defaultdict
import asyncio
#b = Button(description='HideShow')
import chess.engine
from chess import Board 

from calc import Calculator
mycalc=Calculator(engine)

def display_game(pgn):      
    def rev(x):
        a.value=(a.value-1)
    def adv(x):
        a.value=(a.value+1)
    p=chess.pgn.read_game(io.StringIO(pgn))
    gam=p.game()
    current_board= widgets.HTML( str(gam.board()._repr_svg_())) 
    from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider
    
    butl=Layout(width='35px',justify_content='center')
    ba=widgets.Button(description='<',layout=butl)
    ba.on_click(rev)
    ne=widgets.Button(description='>',layout=butl)
    ne.on_click(adv)
    mmax=len(list(gam.mainline_moves()))
    a = widgets.IntSlider(value=0, min=0, max=mmax)
    items = [ba,ne, widgets.Button(description='R',layout=butl),a]
    

    gb2 = widgets.HBox(items,layout=Layout(display='flex',
            flex_flow='row',
            align_items='stretch',
            width='100%'))
    def f2(a):
        #update game progress
        cur=gam
        for k in range(a):
            cur=cur.next()
        display( widgets.HTML(str(cur.board()._repr_svg_())))
    
                
    def f3(a):
        #update stability calc
        cur=gam
        for k in range(a):
            cur=cur.next()
        
        iswhite= (a%2 ==0)
        
        curb=cur.board()
        #display(list(curb.generate_legal_moves()))
        mycalc.print_stats(curb,iswhite)
    
    return VBox([gb2,widgets.interactive_output(f2,{'a':a}),widgets.interactive_output(f3,{'a':a})],layout=Layout(
        display='flex',
        flex_flow='column',
        align_items='stretch',
        width='90%'
    ) )

def display_board(fen,iswhite=None):
    b=chess.Board(fen)
    if iswhite is None:
        iswhite= b.turn
    display( widgets.HTML(str(b._repr_svg_())))
    mycalc.print_stats(b,iswhite)
    
    


## Carlsen game

In [2]:
pgn='''[Event "Biel Chess Festival"]
[Site "Biel SUI"]
[Date "2007.08.02"]
[Round "9"]
[White "Magnus Carlsen"]
[Black "Teimour Radjabov"]
[Result "1-0"]
[EventDate "?"]
[ECO "B07"]
[WhiteElo "2710"]
[BlackElo "2747"]
[PlyCount "55"]

1. e4 d6 2. d4 Nf6 3. Nc3 e5 4. Nge2 Nbd7 5. g3 c6 6. Bg2 b5 7. a3 Be7 8. O-O
O-O 9. h3 a5 10. g4 Ba6 11. Ng3 b4 12. Nce2 bxa3 13. Rxa3 d5 14. Re3 dxe4 15.
Re1 Qc7 16. Nf5 Bd8 17. g5 Nd5 18. Rxe4 f6 19. Neg3 g6 20. Nh6+ Kg7 21. dxe5
fxg5 22. e6 Kxh6 23. e7 Qb6 24. exf8=Q+ Nxf8 25. c4 Nf4 26. Qd6 Kg7 27. Bxf4
gxf4 28. Re7+ 1-0'''
display_game(pgn)

## my stupid game

In [94]:
pgn2='''
[Event "Live Chess"]
[Site "Chess.com"]
[Date "2023.12.05"]
[Round "?"]
[White "nevertheless33"]
[Black "andrewmoussa"]
[Result "1-0"]
[ECO "D01"]
[TimeControl "300"]
[EndTime "14:11:45 PST"]
[Termination "nevertheless33 won by resignation"]

1. d4 d5 2. Nc3 Nf6 3. Bg5 Ne4 4. Nxe4 dxe4 5. Qd2 h6 6. Be3 Bf5 7. h3 e6 8. g4
Bg6 9. O-O-O Be7 10. Kb1 O-O 11. Bf4 Nd7 12. Qc3 c6 13. e3 Bf6 14. Ne2 Re8 15.
Ng3 e5 16. dxe5 Nxe5 17. Rxd8 Raxd8 18. Be2 Nc4 19. Qxc4 b5 20. Qxc6 Re6 21. Qc7
Ra6 22. Rd1 Rxd1+ 23. Bxd1 Rb6 24. Bd6 Ra6 25. Qc8+ 1-0'''

In [125]:
display_game(pgn2)

### Debug

In [103]:
g=[-0.1, 0.63, -0.35, 0.28, -0.07, 0.73, 0.41, 0.64, 0.21, 0.46, 0.75, 0.2, 0.39, 0.72, 0.6, 0.49, -0.1, 0.31, 0.5, 0.01, 0.39, 0.18, -0.49, 0.9, 0.7, -0.01, 0.39, -0.64, 0.02, -0.88, -0.6, -0.56, -0.26, -0.07, -0.47, -0.3, -0.26, -0.68, -0.19, -0.43, -0.21, -0.19, -0.66, 0.0, -0.04, -0.15, -0.21, -0.51, -1.2, -0.75, -0.92, 0.02, -0.46, -0.33, -0.57, 0.41, -0.21, -0.53, -0.17, 0.05, -0.29, -1.18, 0.16, -0.49, 0.48, -0.13, -0.03, -0.67, 0.26, 0.09, -0.15, 0.09, -0.14, -1.21, -0.27, 0.57, 0.22, -0.69, -0.29, -0.04, 0.45, -0.39, 0.0, -0.1, 0.17, 0.48, -0.11, 0.15, 0.72, -0.25, 0.31, 0.2, 0.26, 0.32, 0.25, 0.29, 0.29, 0.18, 0.36, 0.16, -0.31, -0.25, 0.86, 0.47, -0.64, 0.02, -0.31, 0.16, -0.7, 0.09, -0.42, 0.15, -0.29, -0.03, 0.39, -0.46, 0.25, -0.09, -0.07, 0.19, -0.15, 0.06, -0.02, -0.04, 0.07, -0.22, -0.4, -0.41, 0.57, 0.2, -0.22, -0.23, -0.14, 0.07, -0.93, -0.39, -0.42, -0.17, 0.01, -0.32, -0.41, 0.04, -0.29, 0.05, -0.21, -0.07, -0.11, -0.53, -0.13, -0.21, -0.46, -0.18, -0.15, -0.77, -0.42, 0.52, 0.08, -0.21, -0.12, 0.55, 0.71, -0.48, 0.51, 0.12, 0.26, 0.17, 0.6, 1.05, 0.03, 0.62, 0.58, 0.52, 0.72, 0.26, 0.52, 0.42, 0.35, 0.64, 0.3, -0.21, 0.24, 1.25, 0.58, 0.2, 0.12, -0.47, -0.02, -0.88, -0.49, -0.51, -0.17, -0.45, -0.34, 0.4, -0.78, -0.05, -1.21, -0.17, -0.13, -0.91, -0.01, -0.16, -0.1, -0.13, -0.47, -0.61, -0.52, 0.44, 0.0, -1.07, -0.38, -0.24, 0.14, -0.76, -0.04, -0.27, -0.26, 0.09, -0.21, -0.36, 0.28, -0.45, -0.18, -0.21, 0.09, 0.09, -0.1, 0.18, -0.04, -0.16, 0.05, -0.04, -1.53, -0.42, 0.3, 0.11, -0.39, 0.07, -0.54, 0.22, -0.64, -0.09, -0.46, -0.29, -0.03, -0.38, -0.1, 0.12, -0.43, 0.15, -1.11, -0.01, -0.03, -0.32, 0.16, -0.13, -0.24, -0.12, -0.25, -0.69, -0.47, 0.25, 0.22, -0.53, -0.36, -0.17, 0.13, -0.66, -0.04, -0.37, 0.18, 0.06, -0.21, -0.19, 0.49, -0.58, 0.29, 0.03, -0.05, 0.06, -0.07, 0.1, -0.05, -0.19, 0.01, -0.17, -0.47, -0.47, 0.69, 0.4, -0.49, -0.19, -0.23, 0.13, -0.78, -0.21, -0.38, 0.04, 0.12, -0.13, -0.13, 0.33, -0.54, 0.14, -0.04, -0.16, 0.23, -0.42, 0.02, 0.05, -0.21, -0.08, -0.17, -0.57, -0.56, 0.58, 0.19, -0.52, -0.23, -0.29, 0.57, -0.66, 0.26, -0.06, 0.55, 0.47, -0.03, 0.35, 0.64, -0.1, 0.57, 0.48, 0.48, 0.41, -0.19, 0.54, 0.24, 0.06, 0.39, 0.19, -0.28, 0.89, 0.73, -0.21, 0.18, 0.26, 0.42, 0.7, 0.29, 1.57, 0.5, 0.67, 1.47, 0.89, 0.93, 1.13, 1.16, 1.05, 0.91, 0.82, 1.07, 0.65, 1.34, 1.18, 0.23, 0.7, 1.66, 1.28, 0.51, 1.2, 0.42, 0.4, -0.31, 0.26, 0.0, 0.26, 0.38, 0.27, 0.19, 0.41, -0.21, 0.12, -0.16, 0.12, 0.46, -0.05, 0.28, 0.24, 0.26, 0.36, 0.28, 0.04, -0.03, -0.44, 0.45, 0.04, 0.22, -0.75, -0.56, -0.78, -0.92, -0.33, -0.92, -0.03, 1.0, -0.33, -0.63, -0.4, 0.04, -0.04, -0.84, -0.51, -0.93, -0.19, -0.77, -0.95, -0.79, 0.05, -1.15, -0.46, -0.56, -0.29, -0.03, -0.4, -0.23, -0.2, -0.72, -0.31, -0.21, -0.18, -0.9, 0.0, -0.15, -0.35, -0.19, -0.48, -1.79, -0.79, -0.86, 0.1, -0.85, -0.35, -1.04, -0.27, -1.65, -0.77, -0.82, -1.11, -0.89, -0.67, -1.0, -0.09, -0.92, -1.45, 0.05, 0.32, -0.6, -0.5, -1.07, 0.0, -0.1, -0.79, -0.74, -0.77, -1.06, -1.03, -0.11, -0.16, -0.99, -0.87, 0.2, 0.48, -0.7, 0.35, -0.37, 0.5, 0.38, -0.05, 0.05, -0.06, -0.5, 0.83, -0.05, 0.1, 0.04, 0.08, 0.32, 0.42, -0.06, 0.3, 0.08, -0.21, -0.22, 0.85, 0.19, -0.53, 0.47, -0.28, 0.28, -0.55, -0.21, -0.43, 0.22, 0.2, -0.1, 0.13, 0.18, -0.15, 0.27, -0.09, 0.06, 0.08, -0.05, 0.25, 0.06, -0.14, 0.24, 0.01, -0.29, -0.31, 0.62, 0.36, -0.07]
init=5.75

In [112]:
 gb=np.array(g,dtype='float64')/(init/100) * FRACFACTOR

In [113]:
gb

array([ -3.47826087,  21.91304348, -12.17391304,   9.73913043,
        -2.43478261,  25.39130435,  14.26086957,  22.26086957,
         7.30434783,  16.        ,  26.08695652,   6.95652174,
        13.56521739,  25.04347826,  20.86956522,  17.04347826,
        -3.47826087,  10.7826087 ,  17.39130435,   0.34782609,
        13.56521739,   6.26086957, -17.04347826,  31.30434783,
        24.34782609,  -0.34782609,  13.56521739, -22.26086957,
         0.69565217, -30.60869565, -20.86956522, -19.47826087,
        -9.04347826,  -2.43478261, -16.34782609, -10.43478261,
        -9.04347826, -23.65217391,  -6.60869565, -14.95652174,
        -7.30434783,  -6.60869565, -22.95652174,   0.        ,
        -1.39130435,  -5.2173913 ,  -7.30434783, -17.73913043,
       -41.73913043, -26.08695652, -32.        ,   0.69565217,
       -16.        , -11.47826087, -19.82608696,  14.26086957,
        -7.30434783, -18.43478261,  -5.91304348,   1.73913043,
       -10.08695652, -41.04347826,   5.56521739, -17.04

In [116]:
gc[gc<0]=0

In [122]:
gc= (-1)*np.square(gb) *SIGMA

In [123]:
gc

array([-6.04914934e+01, -2.40090737e+03, -7.41020794e+02, -4.74253308e+02,
       -2.96408318e+01, -3.22359168e+03, -1.01686200e+03, -2.47773157e+03,
       -2.66767486e+02, -1.28000000e+03, -3.40264650e+03, -2.41965974e+02,
       -9.20075614e+02, -3.13587902e+03, -2.17769376e+03, -1.45240076e+03,
       -6.04914934e+01, -5.81323251e+02, -1.51228733e+03, -6.04914934e-01,
       -9.20075614e+02, -1.95992439e+02, -1.45240076e+03, -4.89981096e+03,
       -2.96408318e+03, -6.04914934e-01, -9.20075614e+02, -2.47773157e+03,
       -2.41965974e+00, -4.68446125e+03, -2.17769376e+03, -1.89701323e+03,
       -4.08922495e+02, -2.96408318e+01, -1.33625709e+03, -5.44423440e+02,
       -4.08922495e+02, -2.79712665e+03, -2.18374291e+02, -1.11848771e+03,
       -2.66767486e+02, -2.18374291e+02, -2.63500945e+03, -0.00000000e+00,
       -9.67863894e+00, -1.36105860e+02, -2.66767486e+02, -1.57338374e+03,
       -8.71077505e+03, -3.40264650e+03, -5.12000000e+03, -2.41965974e+00,
       -1.28000000e+03, -

In [119]:
gc

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [79]:
display_board('r2Rr1k1/pp3pp1/2p2bbp/4n3/4pBP1/2Q1P1NP/PPP2P2/1K3B1R b - - 0 17',True)

HTML(value='<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" viewBox="0 0 39…

score: -3.72
stability factor: 38.85%
num of reasonable moves: 14
max(score) of reasonable: 2.25
min(score)  of reasonable: -0.12
faraction method: True
moves by depth: {1: 2, 2: 14}
